In [ ]:
#from google.colab import drive
import os
import sys
BASE_DIR = os.path.abspath("../")
if BASE_DIR not in sys.path:
    sys.path = [BASE_DIR] + sys.path

#for colab
# drive.mount('/content/drive', force_remount=True)
# BASE_DIR = "/content/drive/MyDrive/loca/"

